- Promediar las vtas de agosto 2019 (201908) como las de julio (201907) y septiembre (201909) para todas las observaciones
- Buscar los 'product_id' que tengan poca hitoria (agrupandolos por product_id y periodo y validar que tengan menos registros que training_trashold), eliminarlos del conjunto, y agregarlos el un dataframe "Predicciones", poniendo product_id junto con una columna "prediccion", que sea la media de las ventas de los periodos
- Aplicar LabelEncoder a las columnas categoricas
- Agrupar los restantes las ventas por periodo, cat1, cat2, cat3, marca y descripcion
- Calcular para estos el ratio de ventas por product_id (para cada grupo de cat1, cat2, cat3, marca y descripcion), guardando esto en un diccionario: cat1, cat2, cat3, marca, descripcion, product_id y ratio

----
- Agrupar las ventas por periodo, cat1, cat2, cat3, marca, descripcion y customer_id. Sumarizando los valores de las columnas cust_request_qty, cust_request_tn y tn.
- Aplicar escalers por columna a cada grupo (guardando estos scalers en un diccionario)
- Armar un modelo LSTM para predecir las ventas de cada uno de estos grupos (usando todas las observaciones menos las ultimas 2 para predecir la ultima )
----

- Luego, para cada grupo, hacer las predicciones con su modelo correspondiente (usando todas las observaciones menos las primeras 2). Guardando estas predicciones en un dataframe con la estructura cat1, cat2, cat3, marca, descripcion
- sumarizar las predicciones por cat1, cat2, cat3, marca, descripcion
- para cada cat1, cat2, cat3, marca, descripcion, buscar los product_id en el diccionario de ratios, aplicarlo sobre las predicciones sumarizadas, y armar un dataframe product_id y prediccion
- unificar este dataframe con el "Predicciones"
- guardar este df en un csv

#### Imports

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import joblib

In [ ]:
df = pd.read_csv('../../Datasets/final_dataset_descr.csv', sep='\t')

In [ ]:
df.head()

#### Paso 3: Filtrar y eliminar productos con poca historia


Completamos el dataset con 0 para los producto / cliente que no existen

In [ ]:
product_ids = [200001, 200002, 200003, 200004, 200005, 200006, 200007, 200008, 200009, 200010, 200011, 200012]

In [ ]:
filtered_df = df[df['product_id'].isin(product_ids)]

In [ ]:
df['periodo'] = pd.to_datetime(df['periodo'], format='%Y%m')

# Crear un DataFrame con información de productos (esto puede ser parte del dataset original)
product_info = df[['product_id', 'cat1', 'cat2', 'cat3', 'brand', 'sku_size', 'descripcion']].drop_duplicates()

# Paso 2: Identificar el primer y último mes en que cada cliente compró cada producto
min_max_periods = df.groupby(['customer_id', 'product_id'])['periodo'].agg(['min', 'max']).reset_index()

# Crear una lista para almacenar los DataFrames generados
all_dfs = []

# Generar registros para cada combinación de customer_id y product_id
for _, row in min_max_periods.iterrows():
    customer_id = row['customer_id']
    product_id = row['product_id']
    min_period = row['min']
    max_period = row['max']
    all_periods = pd.date_range(min_period, max_period, freq='MS')
    
    combinations = pd.DataFrame({
        'customer_id': [customer_id] * len(all_periods),
        'product_id': [product_id] * len(all_periods),
        'periodo': all_periods
    })
    
    # Paso 3: Merge con el DataFrame original para identificar las combinaciones faltantes
    merged_df = pd.merge(combinations, df, on=['customer_id', 'product_id', 'periodo'], how='left')
    
    # Completar las combinaciones faltantes con tn = 0
    merged_df['tn'] = merged_df['tn'].fillna(0)
    
    # Paso 4: Rellenar las columnas con valores correspondientes de product_info
    merged_df = pd.merge(merged_df, product_info, on='product_id', how='left', suffixes=('', '_y'))
    
    # Calcular las columnas 'quarter' y 'month' a partir del 'periodo'
    merged_df['quarter'] = merged_df['periodo'].dt.to_period('Q').astype(str).str[-1]
    merged_df['month'] = merged_df['periodo'].dt.month.astype(str).str.zfill(2)
    
    # Rellenar las demás columnas con valores predeterminados si es necesario
    merged_df['plan_precios_cuidados'] = merged_df['plan_precios_cuidados'].fillna(0)
    merged_df['cust_request_qty'] = merged_df['cust_request_qty'].fillna(0)
    merged_df['cust_request_tn'] = merged_df['cust_request_tn'].fillna(0)
    merged_df['close_quarter'] = merged_df['close_quarter'].fillna(0)
    merged_df['age'] = merged_df['age'].fillna(0)
    
    # Agregar el DataFrame resultante a la lista
    all_dfs.append(merged_df)

# Concatenar todos los DataFrames generados
df_full = pd.concat(all_dfs, ignore_index=True)

# Ordenar por customer_id, product_id y periodo
df_full = df_full.sort_values(by=['customer_id', 'product_id', 'periodo'])

# Convertir la columna 'periodo' de vuelta al formato original (YYYYMM)
df_full['periodo'] = df_full['periodo'].dt.strftime('%Y%m')

# Mostrar los resultados
print(df_full)

In [ ]:
training_threshold = 12

# Contar el número de registros por product_id, customer_id y periodo
product_history = df.groupby(['product_id', 'customer_id']).size().reset_index(name='counts')

# Filtrar productos por customer_id con menos registros que el threshold
products_to_keep = product_history[product_history['counts'] >= training_threshold][['product_id', 'customer_id']].drop_duplicates()
df_filtered = df.merge(products_to_keep, on=['product_id', 'customer_id'], how='inner')

# Crear el DataFrame "Predicciones" para productos con poca historia por customer_id
products_to_predict = product_history[product_history['counts'] < training_threshold][['product_id', 'customer_id']].drop_duplicates()
predicciones = df.merge(products_to_predict, on=['product_id', 'customer_id'], how='inner').groupby(['product_id', 'customer_id'])['tn'].mean().reset_index()
predicciones.rename(columns={'tn': 'prediccion'}, inplace=True)


#### Paso 4: Aplicar LabelEncoder a las columnas categóricas


In [ ]:
categorical_cols = ['cat1', 'cat2', 'cat3', 'brand', 'descripcion', 'quarter']

# Aplicar LabelEncoder
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    df_filtered[col] = le.fit_transform(df_filtered[col])
    label_encoders[col] = le


#### Paso 5: Agrupar y calcular el ratio de ventas por product_id


In [ ]:
#TODO: Probar con market share de otros meses

# Agrupar por las columnas relevantes
grouped_sales = df_filtered.groupby(['periodo', 'cat1', 'cat2', 'cat3', 'brand', 'product_id'])['tn'].sum().reset_index()

# Calcular el total de ventas por grupo
group_totals = grouped_sales.groupby(['periodo', 'cat1', 'cat2', 'cat3', 'brand'])['tn'].sum().reset_index()

# Unir para calcular el ratio
ratios = pd.merge(grouped_sales, group_totals, on=['periodo', 'cat1', 'cat2', 'cat3', 'brand'], suffixes=('', '_total'))

# Calcular el ratio
ratios['ratio'] = ratios['tn'] / ratios['tn_total']

# Crear un diccionario de ratios
ratio_dict = ratios.set_index(['cat1', 'cat2', 'cat3', 'brand', 'product_id'])['ratio'].to_dict()


#### Paso 6: Agrupar ventas por periodo, cat1, cat2, cat3, brand, descripcion y customer_id


In [ ]:
# Agrupar y sumarizar
grouped_df = df_filtered.groupby(['periodo', 'cat1', 'cat2', 'cat3', 'brand', 'customer_id', 'quarter', 'month']).agg({
    'cust_request_qty': 'sum',
    'cust_request_tn': 'sum',
    'tn': 'sum'
}).reset_index()


In [ ]:
display(grouped_df)

Aplico DTW para agrupar los registros (series de categorias/clientes similares)

In [ ]:
from tslearn.clustering import TimeSeriesKMeans
from tslearn.preprocessing import TimeSeriesScalerMeanVariance

pivoted_df = grouped_df.pivot_table(index=['cat1', 'cat2', 'cat3', 'brand', 'customer_id'], columns='periodo', values='tn').fillna(0)

# Escalar las series temporales
scaler = TimeSeriesScalerMeanVariance(mu=0., std=1.)  # normalizar las series temporales
scaled_series = scaler.fit_transform(pivoted_df.values)

# Paso 3: Aplicar clustering con DTW
n_clusters = 10  # Número máximo de grupos
model = TimeSeriesKMeans(n_clusters=n_clusters, metric="dtw", max_iter=10, random_state=0)
cluster_labels = model.fit_predict(scaled_series)

# Paso 4: Asignar el número de grupo a cada elemento de grouped_df
pivoted_df['cluster'] = cluster_labels

# Unir el número de grupo con grouped_df
grouped_df = grouped_df.merge(pivoted_df['cluster'], left_on=['cat1', 'cat2', 'cat3', 'brand', 'customer_id'], right_index=True)

# Guardar el resultado en un archivo CSV (opcional)
grouped_df.to_csv('grouped_with_clusters.csv', index=False)

# Mostrar el DataFrame final con los números de grupo
display(grouped_df)

#### Paso 7: Aplicar escalers por columna a cada grupo


In [ ]:
# Crear un diccionario para almacenar los scalers
scalers = {}
scaled_df = grouped_df.copy()

# Aplicar StandardScaler a cada columna de interés
for col in ['cust_request_qty', 'cust_request_tn', 'tn']:
    scaler = StandardScaler()
    scaled_df[col] = scaler.fit_transform(scaled_df[[col]])
    scalers[col] = scaler

# Guardar los scalers para su uso posterior
joblib.dump(scalers, 'scalers.pkl')


#### Paso 8: Armar un modelo LSTM


In [ ]:
from keras.optimizers import SGD, RMSprop, Adam
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense, GRU
from keras.regularizers import l2
from sklearn.model_selection import KFold
import numpy as np
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

def build_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(100, activation='tanh', kernel_regularizer=l2(0.1), return_sequences=True, input_shape=input_shape))
    model.add(Dropout(0.1))
    model.add(LSTM(100, activation='tanh', kernel_regularizer=l2(0.1), return_sequences=True))
    model.add(Dropout(0.1))
    model.add(LSTM(512, activation='tanh', kernel_regularizer=l2(0.1), return_sequences=True))
    model.add(Dropout(0.1))
    model.add(LSTM(512, activation='tanh', kernel_regularizer=l2(0.1), return_sequences=True))
    model.add(Dropout(0.1))
    model.add(LSTM(512, activation='tanh', kernel_regularizer=l2(0.1), return_sequences=True))
    model.add(Dropout(0.1))
    model.add(LSTM(256, activation='tanh', kernel_regularizer=l2(0.1), return_sequences=True))
    model.add(Dropout(0.1))
    model.add(LSTM(128, activation='relu', kernel_regularizer=l2(0.1)))
    model.add(Dropout(0.1))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model

#### Paso 9: Entrenar y predecir con el modelo LSTM para cada grupo


In [ ]:
# display(len(grouped_df.groupby(['cat1', 'cat2', 'cat3', 'brand','customer_id'])))

In [ ]:
models = {}
predictions = []

# Preparar los datos por cluster
for cluster in range(n_clusters):
    cluster_data = grouped_df[grouped_df['cluster'] == cluster] #TODO; Reemplazar por scale_df
    if cluster_data.empty:
        continue

    cluster_data.sort_values(by='periodo', inplace=True)
    
    # Preparar los datos para LSTM
    X, y = [], []
    for key, data in cluster_data.groupby(['cat1', 'cat2', 'cat3', 'brand', 'customer_id']):
        series = data[['cust_request_qty', 'cust_request_tn', 'cat1', 'cat2', 'cat3', 'brand', 'customer_id', 'quarter', 'month', 'tn']].values
        n_steps = 2  # Número de pasos para LSTM
        for i in range(n_steps, len(series)):
            X.append(series[i-n_steps:i])
            y.append(series[i, -1])  # 'tn' es la última columna
    
    X = np.array(X)
    y = np.array(y)
    
    if len(X) == 0 or len(y) == 0:
        continue
    
    # Construir y entrenar el modelo
    model = build_lstm_model((X.shape[1], X.shape[2]))
    model.fit(X, y, epochs=20, verbose=2, batch_size=32, validation_split=0.1)
    models[cluster] = model
    
    # Hacer predicciones
    for key, data in cluster_data.groupby(['cat1', 'cat2', 'cat3', 'brand', 'customer_id']):
        series = data[['cust_request_qty', 'cust_request_tn', 'cat1', 'cat2', 'cat3', 'brand', 'customer_id', 'quarter', 'month', 'tn']].values
        X_pred = series[-n_steps:]  # Tomar los últimos n_steps
        X_pred = np.reshape(X_pred, (1, X_pred.shape[0], X_pred.shape[1]))  # Asegurar la forma correcta para la predicción
        pred = model.predict(X_pred, verbose=2)
        predictions.append([key[0], key[1], key[2], key[3], pred[0][0], key[4]])


#### Paso 10: Sumarizar las predicciones y aplicar ratios


In [32]:
display(summarized_preds)

,cat1,cat2,cat3,brand,prediccion
0,0.0,0.0,4.0,22.0,211.116870
1,0.0,0.0,11.0,22.0,239.073733
2,0.0,0.0,37.0,18.0,221.142498
3,0.0,0.0,37.0,19.0,367.793635
4,0.0,0.0,37.0,22.0,693.782610
...,...,...,...,...,...
121,2.0,7.0,74.0,23.0,293.281043
122,3.0,13.0,7.0,32.0,43.879916
123,3.0,13.0,28.0,32.0,16.346868
124,3.0,13.0,31.0,32.0,18.662593


In [33]:
pred_df = pd.DataFrame(predictions, columns=['cat1', 'cat2', 'cat3', 'brand', 'prediccion', 'customer_id'])
pred_df = pd.concat([pred_df, predicciones])

#TODO; Descomentar cuando modifique lo de arriba para usar scaled_df
# scaler_tn = scalers['tn']
# pred_df['prediccion'] = scaler_tn.inverse_transform(pred_df[['prediccion']])

# Sumarizar las predicciones por grupo
summarized_preds = pred_df.groupby(['cat1', 'cat2', 'cat3', 'brand'])['prediccion'].sum().reset_index()

# Aplicar los ratios para obtener las predicciones finales por product_id
# Crear una lista para almacenar las predicciones finales
final_predictions = []

# Iterar sobre cada fila en summarized_preds
for index, row in summarized_preds.iterrows():
    cat1 = row['cat1']
    cat2 = row['cat2']
    cat3 = row['cat3']
    brand = row['brand']
    prediccion = row['prediccion']
    
    # Buscar los ratios correspondientes en ratio_dict
    for (cat1_dict, cat2_dict, cat3_dict, brand_dict, product_id), ratio in ratio_dict.items():
        if cat1 == cat1_dict and cat2 == cat2_dict and cat3 == cat3_dict and brand == brand_dict:
            # Aplicar el ratio al valor de prediccion
            prediction_adjusted = prediccion * ratio
            # Agregar a la lista de predicciones finales
            final_predictions.append([product_id, prediction_adjusted])

# Crear DataFrame con las predicciones finales
final_predictions_df = pd.DataFrame(final_predictions, columns=['product_id', 'prediccion'])

# Mostrar el DataFrame final con las predicciones ajustadas por ratios
display(final_predictions_df)


,product_id,prediccion
0,20609,211.116870
1,20325,33.006901
2,20266,203.177308
3,20503,2.889524
4,20299,221.142498
...,...,...
650,21170,16.346868
651,21202,11.675472
652,21218,6.987121
653,21192,5.082118


In [35]:
final_predictions_df[['product_id', 'prediccion']].to_csv('predicciones_finales.csv', index=False)
